In [12]:
import moviepy.editor as mp
from moviepy.video.fx.all import crop

def detect_action_frame(frame):
    """
    Detects the region of interest (ROI) where the action is happening.
    Here, a simplistic center crop is performed.
    You can improve this by integrating object detection/tracking algorithms.
    """
    height, width, _ = frame.shape
    roi_width = int(width * 0.5)  # Crop width (50% of original width)
    roi_height = height           # Use full height
    
    # Assuming action is centered; modify if object detection is integrated
    x_center = width // 2
    y_center = height // 2
    
    x1 = max(0, x_center - roi_width // 2)
    x2 = min(width, x_center + roi_width // 2)
    
    return x1, x2, 0, height

def convert_to_vertical(input_video, output_video):
    """
    Converts a horizontal video to a vertical one (9:16) using a 'follow the action' approach.
    Preserves and synchronizes the audio track.
    """
    # Load video and audio using moviepy
    video = mp.VideoFileClip(input_video)
    audio = video.audio
    
    # Get original resolution
    original_width, original_height = video.size
    
    # Target vertical resolution (9:16)
    target_width = 1080
    target_height = 1920
    
    # Calculate the scaling factor to maintain aspect ratio
    scaling_factor = target_width / original_width
    new_height = int(original_height * scaling_factor)
    
    # Resize the video
    resized_video = video.resize(width=target_width)
    
    # Crop dynamically based on the 'follow the action' approach
    frames = []
    for frame in resized_video.iter_frames():
        x1, x2, y1, y2 = detect_action_frame(frame)
        cropped_frame = frame[y1:y2, x1:x2]
        frames.append(cropped_frame)
    
    # Convert back to a moviepy video clip
    action_video = mp.ImageSequenceClip(frames, fps=video.fps)
    
    # Resize the cropped video to the target 9:16 format
    final_video = action_video.resize((target_width, target_height))
    
    # Set the audio of the final video
    final_video = final_video.set_audio(audio)
    
    # Export the video with audio
    final_video.write_videofile(output_video, codec='libx264', audio_codec='aac', fps=24)

# Example usage
input_video = "input.mp4"
output_video = "vertical_video.mp4"
convert_to_vertical(input_video, output_video)


Moviepy - Building video vertical_video.mp4.
MoviePy - Writing audio in vertical_videoTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video vertical_video.mp4



Moviepy - Done !
Moviepy - video ready vertical_video.mp4
